## Thinkful exercise 3.2.5 - Dimensionality reduction using random forest model

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Project description

Data:  2015 data from Lending Club, found at this link:https://www.lendingclub.com/info/download-data.action

Modelling objective: Predict the state of a loan given some information about it

Assignment details: 

1) Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

2) Identify which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

3) Can you [answer the question] without using anything related to payment amount or outstanding principal? How do you know?


### Preprocessing data

In [2]:
y2015 = pd.read_csv(
    '../../Datafiles/unit_3/LoanStats3d.csv',
    skipinitialspace=True,
    header=1,
    nrows = 6500
)

In [3]:
y2015.shape

(6500, 111)

In [4]:
y2015.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139,16000,16000,16000,60 months,14.85%,379.39,C,C5,...,0,2,78.9,0.0,0,2,298100,31329,281300,13400
1,68354783,73244544,9600,9600,9600,36 months,7.49%,298.58,A,A4,...,0,2,100.0,66.7,0,0,88635,55387,12500,75635
2,68466916,73356753,25000,25000,25000,36 months,7.49%,777.55,A,A4,...,0,0,100.0,20.0,0,0,373572,68056,38400,82117
3,68466961,73356799,28000,28000,28000,36 months,6.49%,858.05,A,A2,...,0,0,91.7,22.2,0,0,304003,74920,41500,42503
4,68495092,73384866,8650,8650,8650,36 months,19.89%,320.99,E,E3,...,0,12,100.0,50.0,1,0,38998,18926,2750,18248


In [5]:
y2015.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
6495,68242373,73110090,10000,10000,10000,36 months,11.99%,332.10,C,C1,...,0,1,100.0,100.0,0,0,99529,95055,5100,87929
6496,68122492,72990210,24000,24000,24000,60 months,13.99%,558.32,C,C4,...,0,0,100.0,100.0,0,0,54050,35836,30500,23550
6497,68272377,73140097,30000,30000,30000,36 months,19.89%,1113.23,E,E3,...,0,2,95.0,100.0,0,0,133210,122193,43000,87010
6498,68302587,73170315,14400,14400,14400,36 months,13.44%,488.26,C,C3,...,0,4,94.1,0.0,0,0,247189,76146,5800,84354
6499,68082478,72950197,3600,3600,3600,36 months,11.48%,118.68,B,B5,...,0,2,90.9,100.0,0,0,72236,62660,15700,22098


In [6]:
#remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

In [7]:
y2015.shape

(6498, 111)

#### Analyze columns with null values

In [8]:
#One way to remove columns with more nulls than a specified value
#y2015 = y2015.loc[:, y2015.isnull().mean() < .76]

In [9]:
#Compute proportion of null values 
null_counts = pd.DataFrame(y2015.isna().mean().round(4) * 100, columns = ['null_pcnt'])
null_counts.index.name = 'col_name'
null_counts = null_counts.reset_index()
print(null_counts.loc[null_counts.null_pcnt > 0])

                           col_name  null_pcnt
10                        emp_title       5.48
11                       emp_length       5.39
19                             desc      99.98
21                            title       1.89
28           mths_since_last_delinq      48.09
29           mths_since_last_record      82.01
33                       revol_util       0.05
45                     last_pymnt_d       0.05
47                     next_pymnt_d      17.50
50      mths_since_last_major_derog      70.33
53                 annual_inc_joint      99.48
54                        dti_joint      99.48
55        verification_status_joint      99.48
63               mths_since_rcnt_il       2.37
65                          il_util      12.23
76                   bc_open_to_buy       0.98
77                          bc_util       1.08
80               mo_sin_old_il_acct       2.37
85             mths_since_recent_bc       0.97
86         mths_since_recent_bc_dlq      74.47
87           

In [10]:
#Fill nan with zero in some columns, because that is what is implied for these columns
#Pandas 'startswith' works differently than Python. Python allows a beg and end argument
#num_tl prefix means 'number of accounts'
col_list = y2015.loc[:, y2015.columns.str.startswith('mths_')].columns
col_list = col_list.append(y2015.loc[:, y2015.columns.str.startswith('num_tl_')].columns)
col_list = col_list.append(y2015.loc[:, y2015.columns.str.startswith('mo_sin_')].columns)
col_list = col_list.append(y2015.loc[:, y2015.columns.str.contains('bc_')].columns)
for col in col_list:
    y2015[[col]] = y2015[[col]].fillna(value = 0)

In [11]:
#Review remaining columns with null values 
null_counts = pd.DataFrame(y2015.isna().mean().round(4) * 100, columns = ['null_pcnt'])
null_counts.index.name = 'col_name'
null_counts = null_counts.reset_index()
print(null_counts.loc[null_counts.null_pcnt > 0])

                     col_name  null_pcnt
10                  emp_title       5.48
11                 emp_length       5.39
19                       desc      99.98
21                      title       1.89
33                 revol_util       0.05
45               last_pymnt_d       0.05
47               next_pymnt_d      17.50
53           annual_inc_joint      99.48
54                  dti_joint      99.48
55  verification_status_joint      99.48
65                    il_util      12.23


In [12]:
# Convert interest rate to numeric.
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

In [13]:
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

term
2
grade
7
sub_grade
35
emp_title
3878
emp_length
11
home_ownership
3
verification_status
3
issue_d
1
loan_status
7
pymnt_plan
1
url
6498
desc
1
purpose
12
title
12
zip_code
740
addr_state
49
earliest_cr_line
470
revol_util
1009
initial_list_status
2
last_pymnt_d
13
next_pymnt_d
2
last_credit_pull_d
14
application_type
2
verification_status_joint
3


In [14]:
# Drop other columns with many unique variables
drop_cols_list = ['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'id', 'title', 'last_pymnt_d', 'next_pymnt_d']

In [15]:
y2015 = y2015.drop(columns = drop_cols_list)

In [16]:
y2015 = y2015.dropna(axis = 1)

In [17]:
y2015.head()

col_name,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,home_ownership,annual_inc,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,72868139,16000,16000,16000,60 months,14.85,379.39,C,MORTGAGE,48000.0,...,0,2,78.9,0.0,0,2,298100,31329,281300,13400
1,73244544,9600,9600,9600,36 months,7.49,298.58,A,MORTGAGE,60000.0,...,0,2,100.0,66.7,0,0,88635,55387,12500,75635
2,73356753,25000,25000,25000,36 months,7.49,777.55,A,MORTGAGE,109000.0,...,0,0,100.0,20.0,0,0,373572,68056,38400,82117
3,73356799,28000,28000,28000,36 months,6.49,858.05,A,MORTGAGE,92000.0,...,0,0,91.7,22.2,0,0,304003,74920,41500,42503
4,73384866,8650,8650,8650,36 months,19.89,320.99,E,RENT,55000.0,...,0,12,100.0,50.0,1,0,38998,18926,2750,18248


In [18]:
cols_df = pd.DataFrame(list(y2015.columns.values))

In [19]:
print(cols_df)

                                 0
0                        member_id
1                        loan_amnt
2                      funded_amnt
3                  funded_amnt_inv
4                             term
5                         int_rate
6                      installment
7                            grade
8                   home_ownership
9                       annual_inc
10             verification_status
11                         issue_d
12                     loan_status
13                      pymnt_plan
14                         purpose
15                             dti
16                     delinq_2yrs
17                  inq_last_6mths
18          mths_since_last_delinq
19          mths_since_last_record
20                        open_acc
21                         pub_rec
22                       revol_bal
23                       total_acc
24             initial_list_status
25                       out_prncp
26                   out_prncp_inv
27                  

### Thinkful baseline model

In [20]:
# from sklearn import ensemble
# from sklearn.model_selection import cross_val_score

# rfc = ensemble.RandomForestClassifier()
# X = y2015.drop('loan_status', 1)
# Y = y2015['loan_status']
# X = pd.get_dummies(X)
# X = X.dropna(axis=1)

# cross_val_score(rfc, X, Y, cv=10)

Thinkful commentary:

The score cross validation reports is the accuracy of the tree. Here we're about 98% accurate.

That works pretty well, but there are a few potential problems. Firstly, we didn't really do much in the way of feature selection or model refinement. As such there are a lot of features in there that we don't really need. Some of them are actually quite impressively useless.

There's also some variance in the scores. The fact that one gave us only 93% accuracy while others gave higher than 98 is concerning. This variance could be corrected by increasing the number of estimators. That will make it take even longer to run, however, and it is already quite slow.

### Robin's modeling

In [21]:
y2015.loan_status.value_counts()

Current               5132
Fully Paid             931
Charged Off            206
Late (31-120 days)     155
Late (16-30 days)       46
In Grace Period         22
Default                  6
Name: loan_status, dtype: int64

In [22]:
y2015.grade.value_counts()

B    1948
C    1849
A    1217
D     879
E     438
F     145
G      22
Name: grade, dtype: int64

#### Build some features with meaningful delinquency information

Separate collections_12... into collect_none, collect_1_2, collect_gt2

If months since last delinq = 0, means > = 0, < 30 days. So if zero, set no_delinq = True,  else False.

Separate months since last delinq into recent delinquencies -- delinq_last_12_mo = True if >0, <= 12 else false

Then drop cols in sel_cols list

In [23]:
#Change the recent delinquency parameter in number of months
recent_delinq_param = 12
#Use apply to set no_delinq, recent_delinq flags
y2015['no_delinq'] = np.where(y2015.mths_since_last_delinq == 0, True, False)
y2015['recent_delinq'] = np.where(((y2015.mths_since_last_delinq > 0) & (y2015.mths_since_last_delinq <= recent_delinq_param)), True, False)

In [24]:
#Use apply to set no_delinq
y2015['no_derog'] = np.where(y2015.mths_since_last_major_derog == 0, True, False)
y2015['recent_derog'] = np.where(((y2015.mths_since_last_major_derog > 0) & (y2015.mths_since_last_major_derog <= 12)), True, False)

In [25]:
# Drop columns used to categorize delinquencies
drop_cols_list = ['mths_since_last_major_derog', 'mths_since_last_delinq', ]

In [26]:
assert pd.notnull(y2015).all().all()

In [27]:
# from pandas import ExcelWriter
# from pandas import ExcelFile

# writer = ExcelWriter('../../Datafiles/unit_3/y2015_rmng_cols.xlsx')
# cols_df.to_excel(writer,'cols_df',index=False)
# writer.save()

#### Step 1. Normalize variance and drop columns with low variance

In [28]:
y2015.shape

(6498, 98)

In [29]:
y = y2015.loan_status
y2015 = y2015.drop('loan_status', axis = 1)

In [30]:
num_y2015 = pd.DataFrame(y2015.select_dtypes(include=['float64', 'int64']))

In [31]:
from sklearn.feature_selection import VarianceThreshold 

sel = VarianceThreshold(threshold=0.005)
sel.fit(num_y2015 / num_y2015.mean())

mask = sel.get_support() 
reduced_num_y2015 = num_y2015.loc[:, mask] 
print(reduced_num_y2015.shape)

(6498, 81)


In [32]:
# cols_red_df = pd.DataFrame(list(reduced_num_y2015.columns.values))
# writer_reduced = ExcelWriter('../../Datafiles/unit_3/y2015_reduced_cols.xlsx')
# cols_red_df.to_excel(writer,'cols_red_df',index=False)
# writer.save()

In [33]:
y2015 = pd.get_dummies(y2015)

In [34]:
print(y2015.columns)

Index(['member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate',
       'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths',
       ...
       'last_credit_pull_d_Jan-2017', 'last_credit_pull_d_Jul-2016',
       'last_credit_pull_d_Jun-2016', 'last_credit_pull_d_Mar-2016',
       'last_credit_pull_d_May-2016', 'last_credit_pull_d_Nov-2016',
       'last_credit_pull_d_Oct-2016', 'last_credit_pull_d_Sep-2016',
       'application_type_INDIVIDUAL', 'application_type_JOINT'],
      dtype='object', length=134)


In [35]:
y2015.shape

(6498, 134)

In [36]:
keep_cols = y2015.columns.difference(reduced_num_y2015.columns)
print(keep_cols)
y2015_new = pd.merge(reduced_num_y2015, y2015[keep_cols], left_index=True, right_index=True, how='outer')

Index(['application_type_INDIVIDUAL', 'application_type_JOINT', 'grade_A',
       'grade_B', 'grade_C', 'grade_D', 'grade_E', 'grade_F', 'grade_G',
       'home_ownership_MORTGAGE', 'home_ownership_OWN', 'home_ownership_RENT',
       'initial_list_status_f', 'initial_list_status_w', 'issue_d_Dec-2015',
       'last_credit_pull_d_Apr-2016', 'last_credit_pull_d_Aug-2016',
       'last_credit_pull_d_Dec-2015', 'last_credit_pull_d_Dec-2016',
       'last_credit_pull_d_Feb-2016', 'last_credit_pull_d_Jan-2016',
       'last_credit_pull_d_Jan-2017', 'last_credit_pull_d_Jul-2016',
       'last_credit_pull_d_Jun-2016', 'last_credit_pull_d_Mar-2016',
       'last_credit_pull_d_May-2016', 'last_credit_pull_d_Nov-2016',
       'last_credit_pull_d_Oct-2016', 'last_credit_pull_d_Sep-2016',
       'member_id', 'no_delinq', 'no_derog', 'policy_code', 'purpose_car',
       'purpose_credit_card', 'purpose_debt_consolidation',
       'purpose_home_improvement', 'purpose_house', 'purpose_major_purchase',


In [37]:
y2015_new.head()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,...,purpose_small_business,purpose_vacation,pymnt_plan_n,recent_delinq,recent_derog,term_ 36 months,term_ 60 months,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified
0,16000,16000,16000,14.85,379.39,48000.0,33.18,0,0,33.0,...,0,0,1,False,False,0,1,1,0,0
1,9600,9600,9600,7.49,298.58,60000.0,22.44,0,0,0.0,...,0,0,1,False,False,1,0,1,0,0
2,25000,25000,25000,7.49,777.55,109000.0,26.02,0,1,0.0,...,0,0,1,False,False,1,0,1,0,0
3,28000,28000,28000,6.49,858.05,92000.0,21.60,0,0,42.0,...,0,0,1,False,False,1,0,1,0,0
4,8650,8650,8650,19.89,320.99,55000.0,25.49,0,4,0.0,...,0,0,1,False,False,1,0,0,0,1


In [38]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

X = y2015_new

rfc = ensemble.RandomForestClassifier()

cross_val_score(rfc, X, y, cv=10)

/Users/robin/anaconda3/envs/data_sci/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 6 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/Users/robin/anaconda3/envs/data_sci/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/robin/anaconda3/envs/data_sci/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/robin/anaconda3/envs/data_sci/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0

array([ 0.9617737 ,  0.95865237,  0.96313364,  0.96006144,  0.96159754,
        0.96307692,  0.9690881 ,  0.9690881 ,  0.9675425 ,  0.96136012])

#### I've achieved a score above 90%, but how many features are really necessary?

In [39]:
# Calculate the correlation matrix and take the absolute value
corr_matrix = y2015_new.corr().abs()

# Create a True/False mask and apply it
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
y2015_corr = corr_matrix.mask(mask)

# List column names of highly correlated features (r > 0.95)
to_drop = [c for c in y2015_corr.columns if any(y2015_corr[c] >  0.95)]

# Drop the features in the to_drop list
y2015_reduced = y2015_new.drop(to_drop, axis=1)

print("The reduced dataframe has {} columns.".format(y2015_reduced.shape[1]))

The reduced dataframe has 121 columns.


In [40]:
X = y2015_reduced

cross_val_score(rfc, X, y, cv=10)

/Users/robin/anaconda3/envs/data_sci/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 6 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/Users/robin/anaconda3/envs/data_sci/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/robin/anaconda3/envs/data_sci/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/robin/anaconda3/envs/data_sci/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0

array([ 0.96024465,  0.95865237,  0.95238095,  0.95698925,  0.95238095,
        0.95692308,  0.96445131,  0.95981453,  0.95981453,  0.96599691])

In [46]:
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pipe = Pipeline([
('scaler', StandardScaler()),
('reducer', PCA(n_components=45)), 
('classifier', ensemble.RandomForestClassifier())])

pipe.fit(X)

/Users/robin/anaconda3/envs/data_sci/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/robin/anaconda3/envs/data_sci/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/Users/robin/anaconda3/envs/data_sci/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('reducer', PCA(copy=True, iterated_power='auto', n_components=45, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('classifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [48]:
print(pipe.score(X, y))

0.970298553401


/Users/robin/anaconda3/envs/data_sci/lib/python3.6/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
